In [ ]:
project = 'saga-reisetid-prod-4drk'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen viser gjennomsnittlig estimert hastighet og reisetid for strekningen 'Moholt - Haakon VIIs gate' per dag i juni 2021. Merk at både `reisetider` og `strekninger`-datasettene benyttes her.

In [ ]:
query = f"""
SELECT
    DATE(r.publiseringstidspunkt, 'Europe/Oslo') Dato,
    AVG(r.reisetidHastighetKmPerTime) GjennomsnittHastighetKmT,
    AVG(r.reisetidVarighetSekunder) / 60 GjennomsnittReisetidMinutt
FROM `{project}.standardized.reisetider` r
JOIN `{project}.standardized.strekninger` s
    ON s.id = r.strekningId
WHERE DATE(r.publiseringstidspunkt, 'Europe/Oslo') BETWEEN '2021-06-01' AND '2021-06-30'
    AND s.navn = 'Moholt - Haakon VIIs gate'
    AND r.reisetidType = 'ESTIMERT'
GROUP BY Dato
ORDER BY Dato
"""

print(query)

client.query(query).to_dataframe()

Denne spørringer viser de 10 strekningene med høyest estimert gjennomsnittshastighet i 2021.

In [ ]:
query = f"""
SELECT
    s.navn Strekning,
    AVG(r.reisetidHastighetKmPerTime) GjennomsnittHastighetKmT
FROM `{project}.standardized.reisetider` r
JOIN `{project}.standardized.strekninger` s
    ON s.id = r.strekningId
WHERE DATE(r.publiseringstidspunkt, 'Europe/Oslo') BETWEEN '2021-01-01' AND '2021-12-31'
    AND r.reisetidType = 'ESTIMERT'
GROUP BY s.navn
ORDER BY GjennomsnittHastighetKmT DESC
LIMIT 10
"""

print(query)

client.query(query).to_dataframe()